<a href="https://colab.research.google.com/github/HUGOXBACA/ANALOG-CIRCUIT-DESIGN-ANALYSIS-4030/blob/main/GOOGLE_FELLOWSHIP_RESEARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Rail Grade-Crossing Safety***

August 5th, 2023: Hugo: I have found some Federal DOT public data that seems interesting. Therefore I will be doing some exploratory data analysis on this data set and see what I find.

Link to dataset: [Highway-Rail Grade Crossing Accident Data](https://data.transportation.gov/Railroads/Highway-Rail-Grade-Crossing-Accident-Data-Form-57-/7wn6-i5b9)

In [1]:
!pip install pandas
!pip install sodapy

In [2]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.transportation.gov", None)

#client = Socrata( data.transportation.gov,
 #                 '1hCte9vYL7IpE7pFTGR1cbn8y',
  #                username="hugo.baca4515@coyote.csusb.edu",
   #               password="")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("7wn6-i5b9", limit=20)

# Convert to pandas DataFrame
HRGC_data_raw = pd.DataFrame.from_records(results)


In [3]:
#let's see what we get back
print(HRGC_data_raw)

   railroadcode                               railroadname  year  \
0            SP    Southern Pacific Transportation Company  1986   
1          CAGY      Columbus & Greenville Railway Company  1986   
2          CAGY      Columbus & Greenville Railway Company  1986   
3           SOU                   SOUTHERN RAILWAY COMPANY  1976   
4          GSMR  GULF AND MISSISSIPPI RAILROAD CORPORATION  1986   
5           ICG     Illinois Central Gulf Railroad Company  1986   
6          MSRC              MIDSOUTH RAILROAD CORPORATION  1986   
7           ICG     Illinois Central Gulf Railroad Company  1986   
8           ICG     Illinois Central Gulf Railroad Company  1986   
9           ICG     Illinois Central Gulf Railroad Company  1986   
10          ICG     Illinois Central Gulf Railroad Company  1986   
11          ICG     Illinois Central Gulf Railroad Company  1986   
12         GSMR  GULF AND MISSISSIPPI RAILROAD CORPORATION  1986   
13          ICG     Illinois Central Gulf Railro

In [4]:
from numpy.core.fromnumeric import sort

column_headers = list(HRGC_data_raw.columns.values)

column_headers.sort()

print("The Column Headers:")
for header in column_headers:
  print(header)

The Column Headers:
ampm
cityname
countycode
countyname
crossingilluminated
crossingusersinjured
crossinguserskilled
crossingwarning
crossingwarningcode
crossingwarningexpanded1
crossingwarningexpanded2
crossingwarningexpanded3
crossingwarningexpanded4
crossingwarningexpandedcode1
crossingwarningexpandedcode2
crossingwarningexpandedcode3
crossingwarningexpandedcode4
crossingwarninglocation
crossingwarninglocationcode
date
day
district
drivercondition
driverconditioncode
driverinvehicle
driverpassedvehicle
employeesinjured
employeeskilled
equipmentinvolved
equipmentinvolvedcode
equipmentstruck
equipmentstruckcode
equipmenttype
equipmenttypecode
estimatedrecordedspeedcode
estimatedvehiclespeed
form54filed
gradecrossingid
hazmatinvolvement
hazmatinvolvementcode
highwayname
highwayuser
highwayuseraction
highwayuseractioncode
highwayusercode
highwayuserposition
highwayuserpositioncode
hour
incidentmonth
incidentnumber
incidentyear
maintainenanceincidentnumber
maintenance_parent_railroad_cod

So I extracted the colnames, but now lets try to get each label description as to better understand the measurements.

I found this pdf with explainations:  [HIGHWAY-RAIL GRADE CROSSING ACCIDENT/INCIDENT
DATA FILE STRUCTURE AND FIELD INPUT SPECIFICATIONS](https://data.transportation.gov/api/views/icqf-xf4w/files/531a212f-6b94-4403-9876-a6933a1469aa?download=true&filename=gxirfile_EFFECTIVE_060111%20(24).pdf)

I will try to create a description dictionary.

In [ ]:
#from google.colab import files
#uploaded = files.upload()


In [ ]:
from numpy import NaN
dictionary_data_raw = pd.read_excel('Form_57_Data_Dictionary.xlsx')

dictionary = pd.DataFrame.from_records(dictionary_data_raw)

dictionary.columns.values

dictionary = dictionary.filter(items=['Display Name', 'Description'])

print(dictionary)

In [ ]:
# Create a dictionary to store the corresponding names and descriptions
name_to_description = {}
for index, row in dictionary.iterrows():
    name_to_description[row['Display Name']] = row['Description']

# Get the description for the variable 'report year'
description = name_to_description['Report Year']
print(description)

#Personal Prefence: Allows me to quickly pull descriptions for variable.

In [42]:
#Explore for NA's, missing data, and redunnacy

isnaValues = HRGC_data_raw.isna()
print(isnaValues)

# Check for NA values
missing_values = HRGC_data_raw.isna().sum()

#check for null values
missing_data = HRGC_data_raw.isnull().sum()
print(missing_data)

# Check for missing data
for column in HRGC_data_raw.columns:
    if missing_values[column] > 0:
        print(f"The column {column} has {missing_values[column]} missing values.")

# Check for redundancy
for column in HRGC_data_raw.columns:
    if HRGC_data_raw[column].duplicated().sum() > 0:
        print(f"The column {column} has {HRGC_data_raw[column].duplicated().sum()} duplicate values.")

    railroadcode  railroadname   year  incidentnumber  incidentyear  \
0          False         False  False           False         False   
1          False         False  False           False         False   
2          False         False  False           False         False   
3          False         False  False           False         False   
4          False         False  False           False         False   
5          False         False  False           False         False   
6          False         False  False           False         False   
7          False         False  False           False         False   
8          False         False  False           False         False   
9          False         False  False           False         False   
10         False         False  False           False         False   
11         False         False  False           False         False   
12         False         False  False           False         False   
13    

### Crossing Inventory Data

Found another interesting data set:https://data.transportation.gov/Railroads/Crossing-Inventory-Data-Form-71-Current/m2f8-22s6



In [6]:
#lets try to clean up raw data

# Calculate the percentage of missing values in each column
missing_values = HRGC_data_raw.isna().mean()

# Drop columns with more than 30% missing values
drop_list = [column for column in missing_values.index if missing_values[column] > 0.3]

# Drop the columns
HRGC_data_clean = HRGC_data_raw.drop(columns=drop_list)

print(f"\n The Raw Data Frame has {len(HRGC_data_raw.columns)} labels.")
print(f"\n The Clean Data Frame now has {len(HRGC_data_clean.columns)} labels.")
print("\n Count of labels dropped: \n", len(HRGC_data_raw.columns) - len(HRGC_data_clean.columns) )
print("\n Deleted Labels: \n", drop_list)
print("\n Cleaned Data Labels: \n", HRGC_data_clean.columns.values)

HRGC_data_clean.info()



 The Raw Data Frame has 112 labels.

 The Clean Data Frame now has 100 labels.

 Count of labels dropped: 
 12

 Deleted Labels: 
 ['crossingwarningexpandedcode2', 'crossingwarningexpanded2', 'crossingwarningcode', 'crossingwarning', 'maintenance_railroad_class', 'maintenance_railroad_smt_grouping', 'reporting_railroad_company_grouping', 'maintenance_railroad_grouping', 'crossingwarningexpandedcode3', 'crossingwarningexpanded3', 'crossingwarningexpandedcode4', 'crossingwarningexpanded4']

 Cleaned Data Labels: 
 ['railroadcode' 'railroadname' 'year' 'incidentnumber' 'incidentyear'
 'incidentmonth' 'maintenancerailroadcode' 'maintenancerailroadname'
 'maintainenanceincidentnumber' 'maintenanceincidentyear'
 'maintenanceincidentmonth' 'gradecrossingid' 'date' 'month' 'day' 'hour'
 'minute' 'ampm' 'time' 'neareststation' 'countycode' 'countyname'
 'statecode' 'statename' 'cityname' 'highwayname' 'publicprivatecode'
 'publicprivate' 'highwayusercode' 'highwayuser' 'estimatedvehiclespeed'


In [7]:

# importing the modules
from IPython.display import display

#HRGC_data_clean.style
#display(HRGC_data_clean)

from tabulate import tabulate
print(tabulate(HRGC_data_clean, headers = 'keys', tablefmt = 'psql'))
print(HRGC_data_clean.dtypes)

+----+----------------+-------------------------------------------+--------+------------------+----------------+-----------------+---------------------------+-------------------------------------------+--------------------------------+---------------------------+----------------------------+-------------------+-------------------------+---------+-------+--------+----------+--------+----------+------------------+--------------+--------------+-------------+-------------+--------------+------------------+---------------------+-----------------+-------------------+---------------+-------------------------+------------------------+--------------------+---------------------------+------------------------------+-------------------------+------------------------+---------------------------+-----------------------+---------------------------------------+-------------------------+---------------------+---------------+------------------+--------------+------------------------+--------------------

In [ ]:
#lets gather all number type data
from tabulate import tabulate

test = HRGC_data_clean['trainspeed']
print(len(test))
test = pd.to_numeric(test, errors='coerce')
import matplotlib.pyplot as plt
test.plot()
plt.show()

HRGC_data_clean.plot.scatter(y="passengerskilled", x="numberofcars", alpha=0.5)
HRGC_data_clean.plot.scatter(y="passengerskilled", x="trainspeed", alpha=0.5)
HRGC_data_clean.plot.scatter(y="passengerskilled", x="passengersinjured", alpha=0.5)
HRGC_data_clean.plot.scatter(y="passengerskilled", x="passengerskilled", alpha=0.5)



In [46]:
print(set(HRGC_data_clean.railroadname))
print(len(set(HRGC_data_clean.railroadname)))

print(set(HRGC_data_clean.maintenancerailroadname))
print(len(set(HRGC_data_clean.maintenancerailroadname)))

HRGC_data_clean

{'MIDSOUTH RAILROAD CORPORATION', 'Columbus & Greenville Railway Company', 'Illinois Central Gulf Railroad Company', 'Southern Pacific Transportation Company', 'GULF AND MISSISSIPPI RAILROAD CORPORATION', 'SOUTHERN RAILWAY COMPANY'}
6
{nan, 'MIDSOUTH RAILROAD CORPORATION', 'Columbus & Greenville Railway Company', 'Illinois Central Gulf Railroad Company', 'Southern Pacific Transportation Company', 'GULF AND MISSISSIPPI RAILROAD CORPORATION'}
6


In [ ]:
#lets make a corr plot of all vars in the dataframe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = HRGC_data_clean

corr = data.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(data.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(data.columns)
ax.set_yticklabels(data.columns)
plt.show()

In [ ]:
!pip install pandas
!pip install sodapy

In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.transportation.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.transportation.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("m2f8-22s6", limit=2000)

# Convert to pandas DataFrame
Crossing_Inventory_Data_raw__df = pd.DataFrame.from_records(results)

In [ ]:
print(Crossing_Inventory_Data_raw__df)

In [ ]:
from numpy.core.fromnumeric import sort

column_headers_2 = list(Crossing_Inventory_Data_raw__df.columns.values)

column_headers_2.sort()

print("The Column Headers:")
for header in column_headers_2:
  print(header)

In [ ]:
#Explore for NA's, missing data, and redunnacy

isnaValues = Crossing_Inventory_Data_raw__df.isna()
print(isnaValues)

# Check for NA values
missing_values = Crossing_Inventory_Data_raw__df.isna().sum()

#check for null values
missing_data = Crossing_Inventory_Data_raw__df.isnull().sum()
print(missing_data)

# Check for missing data
for column in Crossing_Inventory_Data_raw__df.columns:
    if missing_values[column] > 0:
        print(f"The column {column} has {missing_values[column]} missing values.")

# Check for redundancy
for column in Crossing_Inventory_Data_raw__df.columns:
    if Crossing_Inventory_Data_raw__df[column].duplicated().sum() > 0:
        print(f"The column {column} has {Crossing_Inventory_Data_raw__df[column].duplicated().sum()} duplicate values.")

In [ ]:
#lets try to clean up raw data

# Calculate the percentage of missing values in each column
missing_values = Crossing_Inventory_Data_raw__df.isna().mean()

# Drop columns with more than 30% missing values
drop_list = [column for column in missing_values.index if missing_values[column] > 0.3]

# Drop the columns
CID_data_clean = Crossing_Inventory_Data_raw__df.drop(columns=drop_list)

print(f"\n The Raw Data Frame has {len(Crossing_Inventory_Data_raw__df.columns)} labels.")
print(f"\n The Clean Data Frame now has {len(Crossing_Inventory_Data_raw__df.columns)} labels.")
print("\n Count of labels dropped: \n", len(Crossing_Inventory_Data_raw__df.columns) - len(CID_data_clean.columns) )
print("\n Deleted Labels: \n", drop_list)
print("\n Cleaned Data Labels: \n", CID_data_clean.columns.values)

In [ ]:
# importing the modules
from IPython.display import display

#HRGC_data_clean.style
display(CID_data_clean)

In [ ]:
from tabulate import tabulate
print(tabulate(CID_data_clean, headers = 'keys', tablefmt = 'psql'))
print(CID_data_clean.dtypes)

In [ ]:
print(CID_data_clean.columns)

test2 = CID_data_clean['trainsperweek']
print(len(test))
test = pd.to_numeric(test, errors='coerce')
import matplotlib.pyplot as plt
test.plot()
plt.show()

#CID_data_clean.plot.scatter(y="compower", x="trainsperweek", alpha=0.5)

Index(['revisiondate', 'railroadcode', 'railroadname',
       'reportingagencytypecode', 'reportingagencytype', 'reasoncode',
       'reasondescription', 'crossingid', 'crossingidsuffix',
       'reportingagencycode',
       ...
       'highwaypaved', 'trackrundownstreet', 'highwaynear',
       'smallestcrossinganglecode', 'compower', 'highwaysystemcode',
       'highwaysystem', 'roadatcrossingtypecode', 'roadatcrossingtype',
       'crossingonstatehighwaysystem'],
      dtype='object', length=128)


KeyError: ignored

### Rail Equipment Accident/Incident Data (Form 54)


In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.transportation.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.transportation.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("85tf-25kj", limit=2000)

# Convert to pandas DataFrame
REID = pd.DataFrame.from_records(results)

In [ ]:
from numpy.core.fromnumeric import sort

column_headers_2 = list(REID.columns.values)

column_headers_2.sort()

print("The Column Headers:")
for header in column_headers_2:
  print(header)

  #Explore for NA's, missing data, and redunnacy

isnaValues = REID.isna()
print(isnaValues)

# Check for NA values
missing_values = REID.isna().sum()

#check for null values
missing_data = REID.isnull().sum()
print(missing_data)

# Check for missing data
for column in REID.columns:
    if missing_values[column] > 0:
        print(f"The column {column} has {missing_values[column]} missing values.")

# Check for redundancy
for column in REID.columns:
    if REID[column].duplicated().sum() > 0:
        print(f"The column {column} has {REID[column].duplicated().sum()} duplicate values.")

#lets try to clean up raw data

# Calculate the percentage of missing values in each column
missing_values = REID.isna().mean()

# Drop columns with more than 30% missing values
drop_list = [column for column in missing_values.index if missing_values[column] > 0.3]

# Drop the columns
REID_data_clean = REID.drop(columns=drop_list)

print(f"\n The Raw Data Frame has {len(REID.columns)} labels.")
print(f"\n The Clean Data Frame now has {len(REID.columns)} labels.")
print("\n Count of labels dropped: \n", len(REID.columns) - len(REID_data_clean.columns) )
print("\n Deleted Labels: \n", drop_list)
print("\n Cleaned Data Labels: \n", REID_data_clean.columns.values)

In [ ]:
display(REID_data_clean)

NameError: ignored

### Rail Equipment Accident/Incident Data (Form 54) Subset – Unique Train Accidents (Not at Grade Crossings)

In [ ]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.transportation.gov", None)

results = client.get("byy5-w977", limit=2000)

REID_NON_GRADE = pd.DataFrame.from_records(results)

In [ ]:
from numpy.core.fromnumeric import sort

column_headers_2 = list(REID_NON_GRADE.columns.values)

column_headers_2.sort()

print("The Column Headers:")
for header in column_headers_2:
  print(header)

  #Explore for NA's, missing data, and redunnacy

isnaValues = REID_NON_GRADE.isna()
print(isnaValues)

# Check for NA values
missing_values = REID_NON_GRADE.isna().sum()

#check for null values
missing_data = REID_NON_GRADE.isnull().sum()
print(missing_data)

# Check for missing data
for column in REID_NON_GRADE.columns:
    if missing_values[column] > 0:
        print(f"The column {column} has {missing_values[column]} missing values.")

# Check for redundancy
for column in REID_NON_GRADE.columns:
    if REID_NON_GRADE[column].duplicated().sum() > 0:
        print(f"The column {column} has {REID_NON_GRADE[column].duplicated().sum()} duplicate values.")

#lets try to clean up raw data

# Calculate the percentage of missing values in each column
missing_values = REID_NON_GRADE.isna().mean()

# Drop columns with more than 30% missing values
drop_list = [column for column in missing_values.index if missing_values[column] > 0.3]

# Drop the columns
REID_data_clean = REID_NON_GRADE.drop(columns=drop_list)

print(f"\n The Raw Data Frame has {len(REID_NON_GRADE.columns)} labels.")
print(f"\n The Clean Data Frame now has {len(REID_NON_GRADE.columns)} labels.")
print("\n Count of labels dropped: \n", len(REID_NON_GRADE.columns) - len(REID_data_clean.columns) )
print("\n Deleted Labels: \n", drop_list)
print("\n Cleaned Data Labels: \n", CID_data_clean.columns.values)

In [ ]:
display(REID_data_clean)

,reportingrailroadcode,reportingrailroadname,year,accidentnumber,url,accidentyear,accidentmonth,maintenancerailroadcode,maintenancerailroadname,maintenanceaccidentnumber,...,reporting_railroad_holding_company,maintenance_parent_railroad_code,maintenance_parent_railroad_name,maintenance_railroad_holding_company,trackclass,equipment_type_code,equipmenttype,reporting_railroad_smt_grouping,maintenance_railroad_smt_grouping,division
0,ASAB,Atlanta & Saint Andrews Bay Railway Company,1978,29878,{'url': 'https://safetydata.fra.dot.gov/Office...,78,05,ASAB,Atlanta & Saint Andrews Bay Railway Company,29878,...,Genesee & Wyoming,BAYL,"BAY LINE RAILROAD, LLC",Genesee & Wyoming,NaN,NaN,NaN,NaN,NaN,NaN
1,ARR,Alaska Railroad Corporation,1976,T111,{'url': 'https://safetydata.fra.dot.gov/Office...,76,12,ARR,Alaska Railroad Corporation,T111,...,Not Assigned,ARR,Alaska Railroad Corporation,Not Assigned,1,1,Freight Train,SMT-7 - Commuter West,SMT-7 - Commuter West,NaN
2,ATSF,"Atchison, Topeka & Santa Fe Railway Company",1978,15078103,{'url': 'https://safetydata.fra.dot.gov/Office...,78,07,ATSF,"Atchison, Topeka & Santa Fe Railway Company",15078103,...,BNSF Railway Company,BNSF,BNSF Railway Company,BNSF Railway Company,2,6,Cut of cars,SMT-5 - BNSF,SMT-5 - BNSF,COLORADO
3,BM,Boston & Maine Corporation,1975,750010,{'url': 'https://safetydata.fra.dot.gov/Office...,75,01,BM,Boston & Maine Corporation,750010,...,Not Assigned,GRS,Pan Am Railways/Guilford System,Not Assigned,3,1,Freight Train,NaN,NaN,NEW ENGLAND
4,BN,Burlington Northern Railroad Company,1979,CH338,{'url': 'https://safetydata.fra.dot.gov/Office...,79,02,BN,Burlington Northern Railroad Company,CH338,...,BNSF Railway Company,BNSF,BNSF Railway Company,BNSF Railway Company,1,7,Yard/switching,SMT-5 - BNSF,SMT-5 - BNSF,CHICAGO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,ARZC,Arizona & California Railroad Company,2000,20D101,{'url': 'https://safetydata.fra.dot.gov/Office...,00,01,ARZC,Arizona & California Railroad Company,20D101,...,Genesee & Wyoming,ARZC,Arizona & California Railroad Company,Genesee & Wyoming,3,1,Freight Train,SMT-2 - Short Line East,SMT-2 - Short Line East,CADIZ
1996,ATK,Amtrak (National Railroad Passenger Corporation),2003,076834,{'url': 'https://safetydata.fra.dot.gov/Office...,03,02,ATK,Amtrak (National Railroad Passenger Corporation),076834,...,Amtrak,ATK,Amtrak (National Railroad Passenger Corporation),Amtrak,1,8,Light loco(s),"SMT-1 - Amtrak, Commuter East","SMT-1 - Amtrak, Commuter East",MAD
1997,ATSF,"Atchison, Topeka & Santa Fe Railway Company",1991,010791109,{'url': 'https://safetydata.fra.dot.gov/Office...,91,07,ATSF,"Atchison, Topeka & Santa Fe Railway Company",010791109,...,BNSF Railway Company,BNSF,BNSF Railway Company,BNSF Railway Company,NaN,8,Light loco(s),SMT-5 - BNSF,SMT-5 - BNSF,EASTERN
1998,ALS,Alton & Southern Railway,2011,091411,{'url': 'https://safetydata.fra.dot.gov/Office...,11,09,ALS,Alton & Southern Railway,091411,...,Union Pacific Railroad Company,UP,Union Pacific Railroad Company,Union Pacific Railroad Company,1,6,Cut of cars,SMT-8 - Short Line West,SMT-8 - Short Line West,NaN


### Injury/Illness Summary - Operational Data (Form 55)


In [ ]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.transportation.gov", None)
results = client.get("m8i6-zdsy", limit=2000)

# Convert to pandas DataFrame
illnessSummary = pd.DataFrame.from_records(results)

In [ ]:
from numpy.core.fromnumeric import sort

column_headers_2 = list(illnessSummary.columns.values)

column_headers_2.sort()

print("The Column Headers:")
for header in column_headers_2:
  print(header)

  #Explore for NA's, missing data, and redunnacy

isnaValues = illnessSummary.isna()
print(isnaValues)

# Check for NA values
missing_values = illnessSummary.isna().sum()

#check for null values
missing_data = illnessSummary.isnull().sum()
print(missing_data)

# Check for missing data
for column in illnessSummary.columns:
    if missing_values[column] > 0:
        print(f"The column {column} has {missing_values[column]} missing values.")

# Check for redundancy
for column in illnessSummary.columns:
    if illnessSummary[column].duplicated().sum() > 0:
        print(f"The column {column} has {illnessSummary[column].duplicated().sum()} duplicate values.")

#lets try to clean up raw data

# Calculate the percentage of missing values in each column
missing_values = illnessSummary.isna().mean()

# Drop columns with more than 30% missing values
drop_list = [column for column in missing_values.index if missing_values[column] > 0.3]

# Drop the columns
illnessSummary_data_clean = illnessSummary.drop(columns=drop_list)

print(f"\n The Raw Data Frame has {len(illnessSummary.columns)} labels.")
print(f"\n The Clean Data Frame now has {len(illnessSummary.columns)} labels.")
print("\n Count of labels dropped: \n", len(illnessSummary.columns) - len(illnessSummary_data_clean.columns) )
print("\n Deleted Labels: \n", drop_list)
print("\n Cleaned Data Labels: \n", illnessSummary_data_clean.columns.values)

### Injury/Illness Summary - Casualty Data (Form 55a)

In [ ]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.transportation.gov", None)

results = client.get("rash-pd2d", limit=2000)

# Convert to pandas DataFrame
illnessSummary_Casualty = pd.DataFrame.from_records(results)

In [ ]:
from numpy.core.fromnumeric import sort

column_headers_2 = list(illnessSummary_Casualty.columns.values)

column_headers_2.sort()

print("The Column Headers:")
for header in column_headers_2:
  print(header)

#Explore for NA's, missing data, and redunnacy

isnaValues = illnessSummary_Casualty.isna()
print(isnaValues)

# Check for NA values
missing_values = illnessSummary_Casualty.isna().sum()

#check for null values
missing_data = illnessSummary_Casualty.isnull().sum()
print(missing_data)

# Check for missing data
for column in illnessSummary_Casualty.columns:
    if missing_values[column] > 0:
        print(f"The column {column} has {missing_values[column]} missing values.")

# Check for redundancy
for column in illnessSummary_Casualty.columns:
    if illnessSummary_Casualty[column].duplicated().sum() > 0:
        print(f"The column {column} has {illnessSummary_Casualty[column].duplicated().sum()} duplicate values.")

#lets try to clean up raw data

# Calculate the percentage of missing values in each column
missing_values = illnessSummary_Casualty.isna().mean()

# Drop columns with more than 30% missing values
drop_list = [column for column in missing_values.index if missing_values[column] > 0.3]

# Drop the columns
illnessSummary_Casualty_data_clean = illnessSummary_Casualty.drop(columns=drop_list)

print(f"\n The Raw Data Frame has {len(illnessSummary_Casualty.columns)} labels.")
print(f"\n The Clean Data Frame now has {len(illnessSummary_Casualty.columns)} labels.")
print("\n Count of labels dropped: \n", len(illnessSummary_Casualty.columns) - len(illnessSummary_data_clean.columns) )
print("\n Deleted Labels: \n", drop_list)
print("\n Cleaned Data Labels: \n", illnessSummary_Casualty_data_clean.columns.values)

### HRGC_data_clean EDA

> Indented block
I want a corr plot of deaths and several vars

In [ ]:
from tabulate import tabulate

print(tabulate(HRGC_data_clean, headers = 'keys', tablefmt = 'psql'))



+----+----------------+-------------------------------------------+--------+------------------+----------------+-----------------+---------------------------+-------------------------------------------+--------------------------------+---------------------------+----------------------------+-------------------+-------------------------+---------+-------+--------+----------+--------+----------+------------------+--------------+--------------+-------------+-------------+--------------+------------------+---------------------+-----------------+-------------------+---------------+-------------------------+------------------------+--------------------+---------------------------+------------------------------+-------------------------+------------------------+---------------------------+-----------------------+---------------------------------------+-------------------------+---------------------+---------------+------------------+--------------+------------------------+--------------------

In [ ]:
test_vars = [HRGC_data_clean.estimatedvehiclespeed, HRGC_data_clean.numberofcars, HRGC_data_clean.trainspeed, HRGC_data_clean.crossinguserskilled, HRGC_data_clean.crossingusersinjured, HRGC_data_clean.vehicledamagecost,HRGC_data_clean.employeeskilled,HRGC_data_clean.employeesinjured    ]

target_vars = HRGC_data_clean["estimatedvehiclespeed"]
print(target_vars)

0       0
1      45
2     NaN
3      25
4      25
5       0
6      27
7       2
8     NaN
9       0
10      2
11      7
12     45
13     40
14      5
15      0
16     10
17      5
18      5
19     60
Name: estimatedvehiclespeed, dtype: object
